In [4]:
!pip install -U ultralytics opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.4 MB/s eta 0:00:00


In [2]:
import yaml
import os

dataset_root = "/content/drive/MyDrive/archive(4)/cricket_ball_data"

data_yaml = {
    "path": dataset_root,
    "train": "train/images",
    "val": "valid/images",
    "nc": 1,
    "names": ["ball"]
}

yaml_path = os.path.join(dataset_root, "data.yaml")

with open(yaml_path, "w") as f:
    yaml.dump(data_yaml, f)

print("data.yaml created at:", yaml_path)


data.yaml created at: /content/drive/MyDrive/archive(4)/cricket_ball_data/data.yaml


In [8]:
model = YOLO('yolov8s.pt')   # pretrained weights


In [7]:
from ultralytics import YOLO
import ultralytics

# Verify environment (Python, Torch, CUDA/GPU)
ultralytics.checks()

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.6/112.6 GB disk)


In [ ]:
model.train(
    data="/content/drive/MyDrive/archive(4)/cricket_ball_data/data.yaml",
    epochs=100,
    imgsz=960,          # higher resolution for small ball
    batch=8,
    device=0,           # GPU
    workers=2,
    project="cricket_ball_training",
    name="yolov8_cricket_ball",
    exist_ok=True
)


In [9]:
trained_model = YOLO("/content/best (5).pt")



In [30]:
import pandas as pd
import cv2

In [29]:
for i in range(2,16):
  path = f"/content/{i}.mov"
  results = trained_model(path, stream=True)

  ball_coords = []
  frame_id = 0

  for r in results:
      detected = 0   # visibility flag (default = not visible)

      if r.boxes is not None and len(r.boxes) > 0:
          for box in r.boxes:
              x1, y1, x2, y2 = box.xyxy[0].tolist()
              cx = (x1 + x2) / 2
              cy = (y1 + y2) / 2

              detected = 1  # ball detected in this frame
              ball_coords.append([frame_id, cx, cy, detected])
              break  # only one ball, so break

      # If ball not detected, still log the frame
      if detected == 0:
          ball_coords.append([frame_id, None, None, detected])

      frame_id += 1

  df = pd.DataFrame(ball_coords, columns=["frame", "x", "y","flag"])
  file_name = f"cricket_ball_coordinates_{i}.csv"
  df.to_csv(file_name, index=False)

  df.head()

  trajectory_points = []
  MAX_POINTS = 50

  path = f"/content/{i}.mov"
  cap = cv2.VideoCapture(path)

  width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps    = cap.get(cv2.CAP_PROP_FPS)

  output = f"output_{i}.mp4"
  out = cv2.VideoWriter(
      output,
      cv2.VideoWriter_fourcc(*"mp4v"),
      fps,
      (width, height)
  )

  while cap.isOpened():
      ret, frame = cap.read()
      if not ret:
          break

      results = model(frame, conf=0.25)

      for r in results:
          if r.boxes is not None and len(r.boxes) > 0:
              # assuming only one ball
              box = r.boxes[0]
              x1, y1, x2, y2 = map(int, box.xyxy[0])
              cx = int((x1 + x2) / 2)
              cy = int((y1 + y2) / 2)

              # store center point
              trajectory_points.append((cx, cy))

              # limit trail length
              if len(trajectory_points) > MAX_POINTS:
                  trajectory_points.pop(0)

              # draw bounding box
              cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
              cv2.putText(
                  frame, "ball",
                  (x1, y1 - 10),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                  (0, 255, 0), 2
              )

      # draw trajectory path
      for i in range(1, len(trajectory_points)):
          cv2.line(
              frame,
              trajectory_points[i - 1],
              trajectory_points[i],
              (0, 0, 255),   # red color
              2
          )

      out.write(frame)

  cap.release()
  out.release()



Streaming output truncated to the last 5000 lines.
0: 384x640 1 cricket_ball, 18.9ms
Speed: 4.9ms preprocess, 18.9ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.9ms
Speed: 3.0ms preprocess, 11.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.1ms
Speed: 6.0ms preprocess, 15.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.8ms
Speed: 3.0ms preprocess, 8.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.8ms
Speed: 7.0ms preprocess, 12.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.6ms
Speed: 3.0ms preprocess, 8.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 16.9ms
Speed: 5.8ms preprocess, 16.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detectio

In [31]:
print("project completed")

project completed
